In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load("fr_core_news_md")

In [220]:
liste_gare = pd.read_csv("./liste-des-gares.csv", sep=';')
liste_phrase_test = pd.read_csv("./liste-de-phrases.csv", sep=";")
gares = list(liste_gare["COMMUNE"])
gares = [x.lower() for x in gares]
gares_without_starting = [x[2:] for x in gares if x[0:2] == "l'"]
gares_without_dash = [x.replace("-", " ") for x in gares if "-" in x]
gares = [x[2:] if x[0:2] == "l'" else x for x in gares]

In [221]:
gares_without_starting

['hermitage',
 'isle-jourdain',
 "hospitalet-pres-l'andorre",
 'etang-la-ville',
 'aigle',
 'isle-sur-le-doubs',
 'escarene',
 'isle-sur-la-sorgue',
 'etang-la-ville',
 "isle-d'abeau",
 'argentiere-la-bessee',
 'etang-la-ville',
 'hopital-du-grosbois']

In [194]:
preposition_depart = ["de", "depuis", "à", "sur", "dans", "vers"]
preposition_destination = ["à", "pour", "sur", "dans", "jusque", "jusqu'à", "vers", "de"]
preposition_in_both = ["de", "à", "sur", "dans", "vers"]
preposition = set(preposition_depart + preposition_destination)
verbe_action = ["aller", "partir", "rendre", "rejoindre", "retourner"]
verbe_potentiel_action = ["vouloir", "souhaiter", "essayer", "devoir", "pouvoir", "aimer"]

In [208]:
text = "Je veux aller à la mairie de JOUE LES TOURS depuis Paris"
text = text.lower()
def get_main_data(text):
    for city in gares_without_dash:
        if city in text:
           text = text.replace(city, city.replace(" ", "-"))
    doc = nlp(text)
    position = []
    loc_1, loc_2, nb_loc = [], [], 0
    for w in doc:
        w_text = w.text
        if w.pos_ == "ADP" and w_text in preposition:
            position.append([w_text, w.pos_])
        if w_text in gares:
            if nb_loc == 0:
                if position and position[-1][1] == "ADP":
                    loc_1.append(position[-1])
                loc_1.append([w_text.upper(), "LOC"])
                nb_loc += 1
            else:
                if position and position[-1][1] == "ADP":
                    loc_2.append(position[-1])
                loc_2.append([w_text.upper(), "LOC"])
            position.append([w_text.upper(), "LOC"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_action:
            position.append([w.lemma_, "ACTION"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_potentiel_action:
            position.append([w.lemma_, "POT_ACTION"])
    return [position, loc_1, loc_2]
#print([(w.text, w.label_) for w in doc.ents])
get_main_data(text)

[[['vouloir', 'POT_ACTION'],
  ['aller', 'ACTION'],
  ['à', 'ADP'],
  ['de', 'ADP'],
  ['JOUE-LES-TOURS', 'LOC'],
  ['depuis', 'ADP'],
  ['PARIS', 'LOC']],
 [['de', 'ADP'], ['JOUE-LES-TOURS', 'LOC']],
 [['depuis', 'ADP'], ['PARIS', 'LOC']]]

In [196]:
def is_departure(loc, loc_2):
    if loc[0][1] == "ADP":
        if loc[0][0] not in preposition_in_both:
            if loc[0][0] in preposition_depart:
                return True
    elif loc[0][1] == "LOC" and len(loc_2) == 1:
        return True
    return False

In [197]:
def get_city_name(loc_1, loc_2):
    city_1, city_2 = "", ""
    if len(loc_1) == 2:
        city_1 = loc_1[1][0]
    else:
        city_1 = loc_1[0][0]

    if len(loc_2) == 2:
        city_2 = loc_2[1][0]
    else:
        city_2 = loc_2[0][0]
    return [city_1, city_2]

In [198]:
def is_de_a(loc_1, loc_2):
    prep1 = loc_1[0][0]
    prep2 = loc_2[0][0]
    if prep1 in ["de", "à"] and prep2 in ["de", "à"]:
        return True
    return False

In [216]:
def get_position(text):
    text = text.lower() # faudrait aussi enlever la ponctuation, ça ferait gagner du temps de calcul, pas grand chose certes mais sur des milliers ça peut représenter un gain non négligeable
    [data, loc_1, loc_2] = get_main_data(text)
    if not loc_1 or not loc_2:
        return ["Il manque des villes ou des villes n'ont pas été détecté. Il se peut aussi que la ville n'a pas été détecté parce qu'il n'y a pas de gare"]
    depart, destination = "", ""
    if is_departure(loc_1, loc_2):
        if len(loc_1) == 2:
            depart = loc_1[1][0]
        else:
            depart = loc_1[0][0]
        if len(loc_2) == 2:
            destination = loc_2[1][0]
        else:
            destination = loc_2[0][0]
    elif is_departure(loc_2, loc_1):
        if len(loc_2) == 2:
            depart = loc_2[1][0]
        else:
            depart = loc_2[0][0]
        if len(loc_1) == 2:
            destination = loc_1[1][0]
        else:
            destination = loc_1[0][0]
    elif is_de_a(loc_1, loc_2):
        if loc_1[0][0] == "de":
            depart = loc_1[1][0]
            destination = loc_2[1][0]
        else:
            depart = loc_2[1][0]
            destination = loc_1[1][0]
    else:
        pot_action = False
        action = False
        
        for [w, label] in data:
            if label == "POT_ACTION":
                pot_action = True
                pass
            
            if pot_action:
                if label == "LOC":
                    destination = w
                    cities = get_city_name(loc_1, loc_2)
                    for city in cities:
                        if city != w:
                            depart = city
                    break
                
        if not destination:
            for [w, label] in data:
                if label == "ACTION":
                    action = True
                    pass
                if action:
                    if label == "LOC":
                        destination = w
                        cities = get_city_name(loc_1, loc_2)
                        for city in cities:
                            if city != w:
                                depart = city
                        break
    
    return ["L'" + x if x.lower() in gares_without_starting else x for x in [depart, destination]]
get_position(text)

['PARIS', 'JOUE-LES-TOURS']

In [223]:
nb_passed, nb_no_passed = 0, 0

for i, data in liste_phrase_test.iterrows():
    sentence = data["phrase"]
    destination = data["destination"]
    depart = data["départ"]
    
    if [depart.upper(), destination.upper()] == get_position(sentence):
        nb_passed += 1
    else:
        print(sentence)
        [data, loc_1, loc_2] = get_main_data(sentence)
        print(loc_1)
        print(loc_2)
        nb_no_passed += 1

print("Nombre passé: ", nb_passed)
print("Nombre non passé: ", nb_no_passed)
print("Nombre total: ", nb_passed + nb_no_passed)


Je souhaite visiter un musée de Paris, je pars de strasbourg
[['de', 'ADP'], ['STRASBOURG', 'LOC']]
[]
Je vais visiter un musée de paris, je pars de strasbourg
[['de', 'ADP'], ['PARIS', 'LOC']]
[['de', 'ADP'], ['STRASBOURG', 'LOC']]
Je suis à strasbourg et je retourne à paris
[['à', 'ADP'], ['STRASBOURG', 'LOC']]
[['à', 'ADP'], ['PARIS', 'LOC']]
Nombre passé:  23
Nombre non passé:  3
Nombre total:  26


In [224]:
text_test = "Je souhaite visiter un musée de Paris, je pars de strasbourg"

get_position(text_test)

['PARIS', 'STRASBOURG']